In [32]:
%load_ext autoreload

%autoreload 2

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1 - create a SBM graph and extract the edges list


In [33]:
size = 75
s = size
sizes = [s, s, s]
probs = [[0.45, 0.05, 0.02], [0.05, 0.45, 0.07], [0.02, 0.07, 0.40]]
g = nx.stochastic_block_model(sizes, probs, seed=0)
num_nodes = len(g)
edges_list = []
for source, target in g.edges():
    edges_list.append([source, target])
edges_list = np.array(edges_list)

### 2 - Creates a network superactor and adds to a FURY Scene

In [38]:
from helios.fury.actors import FurySuperActorNetwork
from fury import window

colors_by_block = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
colors = np.array([colors_by_block[i//s] for i in range(len(g))]).astype('float64')
markers = [['o', 's', 'd'][i//s]  for i in range(len(g))]

edge_colors = []
for source, target in g.edges():
    c0 = colors[source//s]
    c1 = colors[target//s]
    edge_colors += [c0, c1]
    
    
centers = np.random.normal(size=(len(g), 3))


network_super_actor = FurySuperActorNetwork(
        positions=centers,
        colors=colors,
        scales=1,
        node_edge_width=0,
        marker=markers,
        edges=edges_list
)

nodes = network_super_actor.nodes.vtk_actor

scene = window.Scene()
scene.add(nodes)
scene.add(network_super_actor.edges.vtk_actor)

In [22]:
network_super_actor.nodes.colors_geo.shape

(900, 3)

In [26]:
network_super_actor.edges.colors_geo.shape, len(edges_list)*2

((8702, 3), 8702)

### 3 - Creates a ShowManager FURY Obj.

In [39]:
from fury import actor, window

window_size = [400, 400]
showm = window.ShowManager(scene, reset_camera=False, size=(
    window_size[0], window_size[1]), order_transparent=False,
)

### 4- Starts the streaming

In [40]:
from fury.stream.widget import Widget

#widget = Widget(showm, encoding='webrtc')
widget = Widget(showm)
widget.start()

created 20 8 1344
url: http://localhost:8489?iframe=1&encoding=mjpeg


In [42]:
widget.return_iframe(300)

### 5 - Start Force-Directed async layout

- ms equals to 100 means that the vtk_positions will be updated at each 100 ms

In [43]:
from helios.core.layout import HeliosFr
layout = HeliosFr(centers,edges_list,showm, network_super_actor)
print('start async helios')
layout.start(100)

start async helios


### 6 - Change some network properties

In [45]:
network_super_actor.nodes.marker_opacity = .8
showm.window.Render()
network_super_actor.nodes.edge_width = .1
network_super_actor.nodes.Uniforms.edgeColor.value = (255, 255, 255)
showm.window.Render()

In [55]:
edge_colors = []
for source, target in g.edges():
    c0 = colors_by_block[source//s]
    c1 = colors_by_block[target//s]
    edge_colors += [c0, c1]
np.array(edge_colors).astype('uint8')

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

In [52]:
network_super_actor.edges.colors = np.array(edge_colors).astype('uint8')
network_super_actor.edges.update()
showm.window.Render()

In [17]:
newColors = np.random.randint(0,256,(len(g),3)).astype('uint8')
network_super_actor.nodes.colors = newColors
network_super_actor.nodes.update()
showm.window.Render()

In [9]:
layout.stop()

In [8]:
widget.stop()